In [1]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

In [2]:
import matplotlib.pyplot as plt
from evallm.experiments.sequence_completion_summary import (
    sequence_completion_results,
    display_prompt as display_prompt_sc,
)

from evallm.experiments.transducer_summary import *
from evallm.experiments.transducer_experiment import *
import evallm

In [3]:
res_4os = run_transducer_experiment(
    model="gpt-4o-2024-05-13",
    num_repeats_per_dfa=30,
    sample_dfa_spec=sample_dfa_spec,
    prompter=prompt_by_key["Basic"]["chat"],
    num_dfas=5,
)

In [4]:
res_o1s = run_transducer_experiment(
    # seed=3,
    model="o1-preview-2024-09-12",
    num_repeats_per_dfa=30,
    sample_dfa_spec=sample_dfa_spec,
    prompter=prompt_by_key["Basic"]["chat"],
    # print_completions=True,
    num_dfas=5,
)

In [5]:
def render_frac(x):
    numer = (np.array(x.success_rate_each) == 1).sum()
    denom = (np.array(x.success_rate_each) != 0.5).sum()
    return f"{numer:.0f}/{denom:.0f}"

In [6]:
pd.DataFrame(
    {
        "gpt-o1": [render_frac(x) for x in res_o1s],
        "gpt-4o": [render_frac(x) for x in res_4os],
        "6gram": [f"{x.kgram_success_rates_each[6 - 2] * 30:.0f}/30" for x in res_4os],
    }
)

,gpt-o1,gpt-4o,6gram
0,25/30,27/30,26/30
1,23/29,24/30,25/30
2,19/30,23/30,28/30
3,22/30,23/30,28/30
4,29/29,30/30,30/30
